In [76]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [77]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [78]:
df_train.drop_duplicates(inplace=True)

In [53]:
df_train[(df_train.userid == 538)].rating.value_counts()

0.0    46
1.0    43
Name: rating, dtype: int64

In [57]:
df_test.userid.max()

127495

In [58]:
df_train_train, df_train_test = train_test_split(df_train, random_state=32, shuffle=True)

In [79]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [54]:
ratings_coo.row

array([102179,   3625,  39495, ...,     88,  41540,    466], dtype=int32)

In [28]:
df_train.rating.value_counts()

1.0    703165
0.0    125770
Name: rating, dtype: int64

In [18]:
len(df_train['overall'].astype(int))

828935

In [21]:
len(df_train)

828935

In [11]:
#Ошибку не дает, но пока непонятно как правильно сформировать и подцепить в модель
#Просто метод формирования матрицы признаков на будущее.
user_feature_coo = sparse.coo_matrix((df_train['overall'].astype(int),
                                     (df_train['userid'],
                                     [0]*len(df_train))))

In [61]:
#Тест тест с датасет
dataset = Dataset()

dataset.fit(
            users=range(df_train.userid.max()+1),
            items=range(df_train.itemid.max()+1) #,
          #  user_features=['overall:1', 'overall:2', 'overall:3', 'overall:4', 'overall:5'],
          #  item_features=[]
)

(interactions, weights) = dataset.build_interactions(
    [(x[11], x[12], x[13]) for x in df_train.values])

In [12]:
user_feature_coo.shape

(127496, 1)

In [63]:
ratings_coo.shape

(127496, 41320)

In [80]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 100 #число параметров вектора 
NUM_EPOCHS = 35 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo,
                  epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [39]:
model.get_user_representations()[0][0:5]

array([0.51621175, 0.6955843 , 0.9193559 , 0.31066826, 0.54708844],
      dtype=float32)

In [33]:
model.predict(1, [0,1,2,3])

array([2.34864521, 2.96497393, 2.16868806, 2.76473236])

In [72]:
preds = model2.predict(df_train_test.userid.values,
                      df_train_test.itemid.values,
                     )

In [73]:
roc_auc_score(df_train_test.rating, preds)

0.9999000562140085

In [81]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [82]:
preds_sub.min(), preds_sub.max()

(0.1041623204946518, 12.132699012756348)

In [83]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [84]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [85]:
df_sub['rating']= normalized_preds

In [86]:
df_sub.to_csv('submission_base_log.csv', index=False)

In [ ]:
#0.76401